In [1]:
from datasets import Dataset
from datetime import datetime, timedelta
import os
import re
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from time import sleep
import torch
from transformers import pipeline

### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 
### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 
### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 

# this basically means "smoke em if you got em" where the "em" is NVIDIA GPU
DEVICE = 0 if torch.cuda.is_available() else -1

SF_USR = os.getenv('SF_USR')
SF_ID  = os.getenv('SF_ID')
SF_WH  = os.getenv('SF_WH')
SF_DB  = os.getenv('SF_DB')
SF_SC  = os.getenv('SF_SC')
SF_RL  = os.getenv('SF_RL')

# connect to database and init a cursor for querying
xct_params = {
    "user":                 SF_USR
   ,"account":              SF_ID
   ,"warehouse":            SF_WH
   ,"database":             SF_DB
   ,"schema":               SF_SC
   ,"role":                 SF_RL
   ,"private_key_file":     os.getenv('PRIVATE_KEY_PATH')
   ,"private_key_file_pwd": os.getenv('PRIVATE_KEY_PASSPHRASE')
   ,"authenticator":        os.getenv('SF_AUTH')
}

SF_XCT = snowflake.connector.connect(**xct_params) #connection object
CSR = SF_XCT.cursor()

In [ ]:
query = f"""select * from {SF_DB}.{SF_SC}.threads_keywords_summarystats""" 
CSR.execute(query)
df = CSR.fetch_pandas_all()



In [3]:
#Import necessary libraries
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import GofChisquarePower, TTestIndPower, NormalIndPower
from scipy.stats import norm

In [ ]:
# Definition of Parameters
CONFIDENCE_LEVEL = 0.95
MARGIN_OF_ERROR = 0.05 
EST_PROPORTION = 0.50
ALPHA = 1 - CONFIDENCE_LEVEL
Z_SCORE = norm.ppf(1- ALPHA / 2)

REQ_SAMPLE_SIZE = np.ceil((Z_SCORE**2 * EST_PROPORTION * (1 - EST_PROPORTION)) / (MARGIN_OF_ERROR**2))


Statistical test: 
Cochran's Formula for Sample Size.

When population is very large compared to sample size.

In [6]:
#check each sentiment and total # of threads. these columns count a thread as 1 if it has >0 mentions wth that sentiment.
keyword_summarystats_cochrans = df.assign(
    POSITIVE_THREADS_SUFFICIENT = df['POSITIVE_THREADS'] >= REQ_SAMPLE_SIZE,
    NEUTRAL_THREADS_SUFFICIENT = df['NEUTRAL_THREADS'] >= REQ_SAMPLE_SIZE,
    NEGATIVE_THREADS_SUFFICIENT = df['NEGATIVE_THREADS'] >= REQ_SAMPLE_SIZE,
    TOTAL_THREADS_SUFFICIENT = df['TOTAL_THREADS'] >= REQ_SAMPLE_SIZE
)
print(keyword_summarystats_cochrans)

         KEYWORD             TOPIC        CATEGORY TOTAL_AUTHORS  \
0          india  global conflicts  issue specific         13490   
1          fraud       culture war           coded          9837   
2          biden       US domestic  issue specific         30503   
3           musk       US domestic  issue specific         34614   
4    afghanistan  global conflicts  issue specific          2650   
..           ...               ...             ...           ...   
99          lgbt       culture war  issue specific         14044   
100         libs       culture war           coded          3804   
101        queer       culture war           coded         10587   
102        jihad                IP  issue specific           933   
103     mangione       US domestic  issue specific           435   

     TOTAL_THREADS TOTAL_POSTS  TOTAL_POSITIVE_POSTS  TOTAL_NEUTRAL_POSTS  \
0            13171       74822                 11155                32638   
1             9329       8720

In [11]:
#Mask for keywords where any column is insufficient
#Sample size is getting huge so I hope this will one day be empty.

sufficiency_cols = ['POSITIVE_THREADS_SUFFICIENT',
                    'NEUTRAL_THREADS_SUFFICIENT',
                    'NEGATIVE_THREADS_SUFFICIENT',
                    'TOTAL_THREADS_SUFFICIENT'
]

insufficient_cols_mask = (~keyword_summarystats_cochrans[sufficiency_cols]).any(axis=1)

insufficient_cols_df = df[insufficient_cols_mask].copy()

print(insufficient_cols_df)

                  KEYWORD             TOPIC        CATEGORY TOTAL_AUTHORS  \
1                   fraud       culture war           coded          9837   
4             afghanistan  global conflicts  issue specific          2650   
5              birth rate           science  issue specific           337   
6                 samaria                IP           coded           100   
9               brainwash       culture war           coded          2861   
10            proletariat          economic           coded           257   
11              class war          economic  issue specific           563   
13                  incel       culture war           coded          2451   
15                  gen x       generations  issue specific           905   
16            problematic       culture war           coded          1723   
19                   acab       US domestic           coded          2788   
20                   doge       US domestic           coded          7688   